In [ ]:
import requests
import json
import yaml
from datetime import datetime  
from datetime import timedelta  
import pandas as pd
from IPython.display import display

import time
import csv

In [ ]:
import urllib2

csv_file = open('C:\<yourpath>\scrapingproj.txt', 'a')

mywriter = csv.writer(csv_file, delimiter='|', quoting=csv.QUOTE_MINIMAL)

mywriter.writerow(['depairportCode', 'depairportLongName', 'dTime', 'dDate', 'arairportCode', 'arairportLongName', \
                'arrTime', 'arrDate', 'durHours', 'durMins', 'stops', 'price', 'fx', 'airline', 'airlineCode', \
                'stopName', 'stopHours', 'stopMins', 'stopType', 'segKey', 'tripKey', 'timeStamp'])

# list of airports on Travelocity
ny = 'New%20York,%20NY%20(NYC-All%20Airports)'
hkg = 'hkg'
dub = 'dxb'
par = 'par'
lax = 'lax'
tyo = 'tyo'
sf = 'San%20Francisco,%20CA%20(QSF%20-%20All%20Airports)'

# building list of destinations
dest = [hkg, dub, par, lax, sf, tyo]

for mix in dest:     # for each destination
    usrtrip = [ny, mix]
    depdt = '03/05/2017'
#     depdt = '02/13/2017'
    depdt = datetime.strptime(depdt, '%m/%d/%Y')
    
    arrdt = '03/12/2017'
#     arrdt = '02/20/2017'
    arrdt = datetime.strptime(arrdt, '%m/%d/%Y')

    # datetime.now() + timedelta(days=1)  

    for dpct in range(0, 8):        # outer loop of depdt. start Sun.
        newdeptdt = depdt + timedelta(days = dpct)
        
        newdepdt = str(newdeptdt)
        newdepdt = newdepdt[5:7] + '/' + newdepdt[8:10] +  '/' + newdepdt[0:4]

        for retct in range(0,8):          # inner loop of retdt
            newretdt = arrdt + timedelta(days = retct + dpct)

            newarrdt = str(newretdt)
            newarrdt = newarrdt[5:7] + '/' + newarrdt[8:10] +  '/' + newarrdt[0:4]

            url = "https://www.travelocity.com/Flights-Search?trip=roundtrip&leg1=from:" + usrtrip[0] + \
                       ",to:" + usrtrip[1] + ",departure:" + newdepdt + "TANYT&leg2=from:" + usrtrip[1] + "," + \
                       "to:" + usrtrip[0] + ",departure:" + newarrdt + "TANYT&" + \
                       "passengers=children:0,adults:1,seniors:0,infantinlap:Y&mode=search"

            print url

            try:
                f = urllib2.urlopen(url)
                ht = f.read()
                f.close()

                for item in ht.split("\n"):              # grab json key
                    if 'originalContinuationId' in item:
                         ht = item.strip()

                tkey = ht.split('>')[1].split('<')[0]

                # grab data in json
                page = requests.get('https://www.travelocity.com/Flight-Search-Paging?c=' + tkey)
                page = page.json()

                for myhash in page['content']['legs'].keys():
                    airportCode =  str(page['content']['legs'][myhash]['departureLocation']['airportCode'])
                    airportLongName = str(page['content']['legs'][myhash]['departureLocation']['airportLongName'])
                    dTime = str(page['content']['legs'][myhash]['departureTime']['time'])
                    dDate = str(page['content']['legs'][myhash]['departureTime']['date'])

                    arrCode = str(page['content']['legs'][myhash]['arrivalLocation']['airportCode'])
                    arrName = str(page['content']['legs'][myhash]['arrivalLocation']['airportLongName'])
                    arrTime = str(page['content']['legs'][myhash]['arrivalTime']['time'])
                    arrDate = str(page['content']['legs'][myhash]['arrivalTime']['date'])
                    # print page['content']['legs'][(q[115])]['timeline']
                    durHours = str(page['content']['legs'][myhash]['duration']['hours'])
                    durMins = str(page['content']['legs'][myhash]['duration']['minutes'])

                    stops = str(page['content']['legs'][myhash]['stops'])
                    price = page['content']['legs'][myhash]['price']['totalPriceAsDecimal']
                    fx = page['content']['legs'][myhash]['price']['currencyCode']
                    airline = str(page['content']['legs'][myhash]['carrierSummary']['airlineName'])
                    airlineCode = str(page['content']['legs'][myhash]['carrierSummary']['airlineCodes'])

                    stopName = str(page['content']['legs'][myhash]['timeline'][1]['airport']['name'])       # odd list indices are layovers. Evens are flights.
                    stopHours = page['content']['legs'][myhash]['timeline'][1]['duration']['hours']
                    stopMins = page['content']['legs'][myhash]['timeline'][1]['duration']['minutes']
                    stopType = str(page['content']['legs'][myhash]['timeline'][1]['type'])

                    print [airportCode, airportLongName, dTime, dDate, arrCode, arrName, \
                                    arrTime, arrDate, durHours, durMins, stops, price, fx, airline, airlineCode, \
                                    stopName, stopHours, stopMins, stopType, str(datetime.now())]

                    mywriter.writerow([airportCode, airportLongName, dTime, dDate, arrCode, arrName, \
                                    arrTime, arrDate, durHours, durMins, stops, price, fx, airline, airlineCode, \
                                    stopName, stopHours, stopMins, stopType, myhash, tkey, str(datetime.now())])

            except:
                pass

csv_file.close()